# Tour of New Harmony Services
## PI 21.4 Solution Demonstration

Outline:
1. Segmented Trajectory Subsetter in support of GEDI and ICESat-2 data
2. Data aggregation: PO.DAAC CONCISE service
3. Cloud Optimized GeoTIFF (COG) reformatting
4. Zarr customized chunking 


For each service demo, include at a minimum:
* Input data support: Data structures and levels supported, Collections tested against the service, Collections currently supported
* Request via Harmony-py: Required/supported service params
* Obtain output S3 URLs
* Open s3 files (e.g. with s3fs and xarray to demonstrate AiP)
* Make a pretty picture / demonstrate transformations performed to input granules



## Import Packages

In [31]:
from harmony import BBox, Client, Collection, Request, LinkType
from harmony.config import Environment
from pprint import pprint
#import s3fs
import boto3
import xarray as xr
import rioxarray
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
import hvplot.xarray
import os
import holoviews as hv

%matplotlib inline

### Create Harmony Client object
First, we need to create a Harmony Client, which is what we will interact with to submit and inspect a data request to Harmony, as well as to retrieve results.

When creating the Client, we need to provide Earthdata Login credentials, which are required to access data from NASA EOSDIS. This basic line below assumes that we have a `.netrc` available. 

In [2]:
harmony_client = Client(env=Environment.UAT)

In [3]:
short_name = 'C1239379702-POCLOUD'
granuleid = 'G1241058382-POCLOUD'

http://{{harmony_host}}/C1239379702-POCLOUD/ogc-api-coverages/1.0.0/collections/EXFatemp/coverage/rangeset?format=image/tiff&granuleId=G1241058382-POCLOUD&forceAsync=true&turbo=true

In [4]:
request = Request(
    collection=Collection(id=short_name),
    granule_id=granuleid,
    variables=['EXFatemp'],
    format='image/tiff'
)

In [5]:
job_id = harmony_client.submit(request)
job_id

'3375a59c-d70c-4015-9c3b-29859aeade0f'

In [6]:
harmony_client.status(job_id)

{'status': 'running',
 'message': 'The job is being processed',
 'progress': 0,
 'created_at': datetime.datetime(2021, 12, 10, 17, 35, 2, 175000, tzinfo=tzlocal()),
 'updated_at': datetime.datetime(2021, 12, 10, 17, 35, 2, 175000, tzinfo=tzlocal()),
 'request': 'https://harmony.uat.earthdata.nasa.gov/C1239379702-POCLOUD/ogc-api-coverages/1.0.0/collections/EXFatemp/coverage/rangeset?forceAsync=true&granuleId=G1241058382-POCLOUD&format=image%2Ftiff',
 'num_input_granules': 1}

In [7]:
harmony_client.wait_for_processing(job_id, show_progress=True)

 [ Processing: 100% ] |###################################################| [|]


In [8]:
data = harmony_client.result_json(job_id)
pprint(data)

{'createdAt': '2021-12-10T17:35:02.175Z',
 'jobID': '3375a59c-d70c-4015-9c3b-29859aeade0f',
 'links': [{'href': 'https://harmony.uat.earthdata.nasa.gov/stac/3375a59c-d70c-4015-9c3b-29859aeade0f/',
            'rel': 'stac-catalog-json',
            'title': 'STAC catalog',
            'type': 'application/json'},
           {'bbox': [-180, -90, 180, 90],
            'href': 'https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-staging/public/asfdataservices/gdal-subsetter/5a052a8c-5313-447e-bcd2-8e3f7e33a8b6/ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO_V4r4_latlon_0p50deg_EXFatemp.tif',
            'rel': 'data',
            'temporal': {'end': '2017-12-16T00:00:00.000Z',
                         'start': '2017-12-15T00:00:00.000Z'},
            'title': 'ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO_V4r4_latlon_0p50deg_EXFatemp.tif',
            'type': 'image/tiff'},
           {'href': 'https://harmony.uat.earthdata.nasa.gov/jobs/3375a59c-d70c-4015

In [9]:
results = harmony_client.result_urls(job_id, link_type=LinkType.s3)
url = list(results)
url

['s3://harmony-uat-staging/public/asfdataservices/gdal-subsetter/5a052a8c-5313-447e-bcd2-8e3f7e33a8b6/ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO_V4r4_latlon_0p50deg_EXFatemp.tif']

In [10]:
creds = harmony_client.aws_credentials()

### Insert the credentials into our `boto3` session and configure our `rasterio` environment for data access

Create a boto3 Session object using your temporary credentials. This Session is used to pass credentials and configuration to AWS so we can interact wit S3 objects from applicable buckets.

In [12]:
session = boto3.Session(aws_access_key_id=creds['aws_access_key_id'], 
                        aws_secret_access_key=creds['aws_secret_access_key'],
                        aws_session_token=creds['aws_session_token'],
                        region_name='us-west-2')

For this exercise, we are going to open up a context manager for the notebook using the `rasterio.env` module to store the required GDAL and AWS configurations we need to access the data in Earthdata Cloud. While the context manager is open (`rio_env.__enter__()`) we will be able to run the open or get  data commands that would typically be executed within a `with` statement, thus allowing us to more freely interact with the data. We’ll close the context (`rio_env.__exit__()`) at the end of the notebook.

GDAL environment variables must be configured to access Earthdata Cloud data assets. Geospatial data access Python packages like `rasterio` and `rioxarray` depend on GDAL, leveraging GDAL's "Virtual File Systems" to read remote files. GDAL has a lot of environment variables that control it's behavior. Changing these settings can mean the difference being able to access a file or not. They can also have an impact on the performance.

In [13]:
rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

## Read in a single COG file

We'll access the S3 object using the [`rioxarray`](https://corteva.github.io/rioxarray/stable/) Python package. The package is an extension of `xarray` and `rasterio`, allowing users to read in and interact with geospatial data using xarray data structures. We will also be leveraging the tight integration between xarray and dask to lazily read in data via the `chunks` parameter. This allows us to connect to the S3 object, reading only metadata, an not load the data into memory until we request it via the `loads()` function. 

In [14]:
url[0]

's3://harmony-uat-staging/public/asfdataservices/gdal-subsetter/5a052a8c-5313-447e-bcd2-8e3f7e33a8b6/ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO_V4r4_latlon_0p50deg_EXFatemp.tif'

In [ ]:
# rds = rioxarray.open_rasterio(cog_url, masked=True, overview_level=4)

In [17]:
da = rioxarray.open_rasterio(url[0], chunks=True)
da

<xarray.DataArray (band: 4, y: 360, x: 720)>
dask.array<open_rasterio-125feb2c42144b189056541b1e954073<this-array>, shape=(4, 360, 720), dtype=uint8, chunksize=(1, 360, 720), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * y            (y) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
    spatial_ref  int64 0
Attributes:
    bandname:       ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO_V...
    long_name:      ('ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO...
    standard_name:  ATM_SURFACE_TEMP_HUM_WIND_PRES_day_mean_2017-12-15_ECCO_V...
    scale_factor:   1.0
    add_offset:     0.0

In [40]:
# da.plot.imshow(rgb="band") ;

In [39]:
da.hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, xaxis=False, yaxis=None)

:RGB   [x,y]   (R,G,B)

#### Example - Reading COGs in Parallel¶

https://corteva.github.io/rioxarray/stable/examples/read-locks.html

# Trajectory Subsetter demo example requests

The scope of this notebook is to provide example requests for the PI 21.4 Harmony services Inspect and Adapt deomonstrations. This will include:

* Variable subsetting.
* Temporal subsetting.
* Bounding box spatial subsetting.
* Polygon spatial subsetting (EDSC instructions).

Distinctions from other services:

* Works on trajectory, rather than swath data.
* Preserves continuity of photon segment indices.
* Configured for Cloud-hosted GEDI L4A test collection.

Demonstrations notes:

To keep total time of the demonstration down, it may be best to skip the variable subset only request. That functionality is demonstrated by all of the other requests.

It is easier to demonstrate success with the spatial subsets, rather than the temporal subset, as the output variables for latitude and longitude are in degrees, which corresponds to be bounding box or shape file specified. The temporal subset is a bit harder to visually interpret, as the `/BEAM0000/delta_time` variable is an amount of time since an epoch.

### Set up imports and variables

In [41]:
from datetime import datetime
from os import replace

from harmony import BBox, Client, Collection, Environment, Request


gedi_l4a = Collection(id='C1242267295-EEDTEST')
file_root_name = 'GEDI_L4A'
variables = ['/BEAM0000/agbd', '/BEAM0000/delta_time', '/BEAM0000/lat_lowestmode', '/BEAM0000/lon_lowestmode']
temporal_range = {'start': datetime(2020, 7, 25, 1, 0, 0), 'stop': datetime(2020, 7, 25, 2, 0, 0)}
bounding_box = BBox(w=10, s=10, e=30, n=30)

harmony_client = Client(env=Environment.UAT)

### Variable subsetting:

The GEDI L4A collection has granules that are at least 1 GB in size. The Trajectory Subsetter provides the capability to perform a variable subset, to reduce the size of the output product. The variable subset will include:

* `/BEAM0000/agbd`, above ground biomass density.
* `/BEAM0000/delta_time`, the temporal coordinate for the science variable.
* `/BEAM0000/lat_lowestmode`, the latitude coordinate for the science variable.
* `/BEAM0000/lon_lowestmode`, the longitude coordinate for the science variable.

Things to check:

* All four requested variables are present in the output.
* Note the approximate size of the variable arrays, for comparison to later requests.

In [9]:
variable_subset_request = Request(collection=gedi_l4a, variables=variables, max_results=1)
variable_subset_job_id = harmony_client.submit(variable_subset_request)
variable_local_file_name = f'{file_root_name}_variable.h5'

print(f'Processing job: {variable_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(variable_subset_job_id, overwrite=True)]:
    print(f'Downloaded: {filename}')
        
replace(filename, variable_local_file_name)
print(f'Saved output to: {variable_local_file_name}')

Processing job: 97321278-751a-4a28-a9d8-8bd68e7feb4d
Downloaded: GEDI04_A_2019220190014_O03710_T02251_02_001_01_subsetted.h5
Saved output to: GEDI_L4A_variable.h5


### Temporal subsetting:

The Trajectory Subsetter accepts a single temporal range. In the request below, the temporal subset has been combined with a variable subset to maintain a small output product.

The temporal range specified is from 2020-07-25T01:00:00 to 2020-07-25T02:00:00.

Things to check in output:

* Only the four requested parameters are present.
* The size of all four variables is the same.
* The size of all four variables is smaller than the variable subset request.

In [10]:
temporal_subset_request = Request(collection=gedi_l4a, temporal=temporal_range, max_results=1, variables=variables)
temporal_subset_job_id = harmony_client.submit(temporal_subset_request)
temporal_local_file_name = f'{file_root_name}_temporal.h5'

print(f'Processing job: {temporal_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(temporal_subset_job_id, overwrite=True)]:
    print(f'Downloaded: {filename}')

replace(filename, temporal_local_file_name)
print(f'Saved output to: {temporal_local_file_name}')

Processing job: 1813080c-2543-445a-9ac7-3253213f108c
Downloaded: GEDI04_A_2020207012314_O09157_T05052_02_001_01_subsetted.h5
Saved output to: GEDI_L4A_temporal.h5


### Bounding box spatial subsetting:

The Trajectory Subsetter accepts a single bounding box. The request below combines a bounding box with a variable subset to maintain a small output product.

The bounding box specified is:

* 10 ≤ latitude (degrees north) ≤ 30
* 10 ≤ longitude (degrees east) ≤ 30

Things to check in output:

* The four requested variables are present in the output.
* The variables all have the same size, which should be smaller than the variable subset only.
* The `/BEAM0000/lat_lowestmode` only has values in the range of the bounding box.
* The `/BEAM0000/lon_lowestmode` only has values in the range of the bounding box.

In [11]:
bbox_subset_request = Request(collection=gedi_l4a, spatial=bounding_box, max_results=1, variables=variables)
bbox_subset_job_id = harmony_client.submit(bbox_subset_request)
bbox_local_file_name = f'{file_root_name}_bbox.h5'

print(f'Processing job: {bbox_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(bbox_subset_job_id, overwrite=True)]:
    print(f'Downloaded: {filename}')
    
replace(filename, bbox_local_file_name)
print(f'Saved output to: {bbox_local_file_name}')

Processing job: ed6016fa-049a-41d5-bd55-271af61f6576
Downloaded: GEDI04_A_2019220190014_O03710_T02251_02_001_01_subsetted.h5
Saved output to: GEDI_L4A_bbox.h5


### Polygon spatial subsetting:

The Trajectory Subsetter can accept a GeoJSON shape file to perform polygon spatial subsetting.

This request is best demonstrated via Earthdata Search Client (EDSC).

* Navigate to EDSC (UAT).
* Load a GeoJSON shape file using the filters in the left toolbar (Egypt shape file should be distributed with this notebook).
* Search for "GEDI L4A EEDTEST".
* Select the collection. Then one of the granules that passes through the Egypt GeoJSON.
* Click "Download (1)".
* In the download customisation form:
  * Select Harmony ("More Info" should show "sds-trajectory-subsetter").
  * Ensure the four variables are selected (to maintain a small output).
  * Mention that the shape file is automatically sent in the Harmony request.
* Click "Download Data".
* Things to check in output:
  * The four requested variables are present in the output file.
  * The variables all have the same size (which should be smaller than the variable subset only).
  * The `/BEAM0000/lat_lowestmode` should be only within Egypt, approximately: 22.0 ≤ latitude (degrees north) ≤ 31.6.
  * The `/BEAM0000/lon_lowestmode` should be only within Egypt, approximately: 24.6 ≤ longitude (degrees east) ≤ 35.8.

## CONCISE aggregation

## Cloud Optimized GeoTIFF (COG) Reformatting

## Zarr Customized Chunking